In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os
from tqdm import tqdm
import cv2

import torch
from lime import lime_image
import shap

from api_standard import ApiClass as oracle
oracle.init()

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_list = [
    './enh/face1_enh.png',
    './enh/face2_enh.png',
    './enh/noface1_enh.png',
    './enh/noface2_enh.png'
]

for i in tqdm(range(1, 31)):
    oracle.lowlight_enhance_by_path(f'./img/extra/{i}.png', f'./enh/{i}.png')
    file_list.append(f'./enh/{i}.png')

print(len(file_list))

100%|██████████| 30/30 [00:01<00:00, 24.32it/s]

34


In [3]:
def read_img(src_path):
    with open(src_path, 'rb') as f:
        img = Image.open(f).convert('RGB')
    img = np.array(img, dtype=np.float64) / 255
    return img

imgs = np.array([read_img(file_path) for file_path in file_list])

In [4]:
def batch_predict(imgs):
    ps = oracle.get_confidence_by_ndarray(np.array(imgs).transpose((0,3,1,2)))
    ps = np.array([1-ps, ps]).transpose((1,0))
    assert len(ps.shape) == 2 and ps.shape[1] == 2
    return ps

In [ ]:
for file_path in file_list:
    img = read_img(file_path)
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(img, batch_predict, top_labels=2, hide_color=0, num_samples=1000, batch_size=50)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-1.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=2, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-2.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-5.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-10.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=15, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-15.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=20, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-20.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=50, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-50.png'), temp[:, :, [2,1,0]]*255)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=100, hide_rest=False)
    cv2.imwrite(file_path.replace('enh','lime').replace('.png','-100.png'), temp[:, :, [2,1,0]]*255)

100%|█████████▉| 997/1000 [01:05<00:00, 33.54it/s]

In [ ]:
# temp, mask = explainer.get_image_and_mask(explainer.top_labels[0], positive_only=False, num_features=20, hide_rest=False)
# plt.imshow(temp)

In [ ]:
# masker = shap.maskers.Image("inpaint_telea", imgs[0].shape)
# shap_explainer = shap.Explainer(batch_predict, masker, output_names=['noface', 'hasface'])
# shap_values = shap_explainer(imgs.copy(), max_evals=1600, batch_size=16)
# shap.image_plot(shap_values)